In [1]:
import sys
import os
import tensorflow as tf
import scipy.io.wavfile as wav
import glob
from python_speech_features import mfcc, logfbank
import numpy as np
from json import load

sys.path.append(os.path.abspath(os.path.join('..')))
from scripts.model_trainer import CTCLossLayer


In [25]:
train_model = tf.keras.models.load_model(
    '../models/stacked-lstm_train.h5', custom_objects={'CTCLossLayer':CTCLossLayer})
test_model = tf.keras.models.load_model('../models/stacked-lstm_predict.h5')


In [15]:
test_model.get_weights()[0]

array([[ 9.8760156e-03,  5.0904090e-03,  2.1440536e-03, ...,
         7.1010218e-05,  2.2191240e-02, -1.1521566e-03],
       [ 1.0374833e-02,  3.7352354e-03,  1.9712982e-04, ...,
         1.3160312e-02,  1.7267222e-02, -3.9844969e-03],
       [ 2.0356745e-02,  4.9347891e-03,  3.3088583e-03, ...,
        -3.6529331e-03,  2.0845376e-02, -1.3723028e-03],
       ...,
       [ 3.2377396e-02,  4.8139649e-03, -3.8627798e-03, ...,
        -3.2257955e-03, -6.0509192e-03, -3.9089015e-03],
       [ 2.3114018e-02,  5.0957259e-03, -6.5801837e-03, ...,
         3.1469143e-03,  2.2563681e-02, -2.6519238e-03],
       [ 2.0118238e-02,  4.4266721e-03, -7.4877911e-03, ...,
         1.0561711e-02,  1.7587548e-02, -4.5149270e-03]], dtype=float32)

In [16]:
train_model.get_weights()[0]

array([[ 1.7607717e-02,  7.8646354e-03, -2.9276735e-03, ...,
        -6.0886052e-03,  2.3322381e-02,  1.0254472e-03],
       [ 1.6778247e-02,  5.9010573e-03, -4.8780143e-03, ...,
         1.4590365e-02,  1.8398089e-02,  1.3089210e-03],
       [ 2.9123748e-02,  8.3491579e-03, -6.0243993e-03, ...,
        -4.4795186e-03,  1.9652918e-02, -5.9251970e-04],
       ...,
       [ 3.3143297e-02,  9.8850746e-03, -7.3817526e-03, ...,
         3.1702055e-03, -1.3194567e-02, -5.8042508e-04],
       [ 2.2572132e-02,  1.0355887e-02, -8.7000532e-03, ...,
         7.5934164e-05,  1.6969645e-02, -9.2280519e-05],
       [ 2.1201307e-02,  1.0082344e-02, -7.5118048e-03, ...,
         8.4226280e-03,  1.2096869e-02,  1.6247890e-03]], dtype=float32)

In [ ]:
# test_model.set_weights(train_model.get_weights())

In [19]:
files_path = glob.glob('../data/train/wav/' + '*.wav')

audio_list = []

for file_name in files_path:
    fs, audio = wav.read(file_name)
    audio_list.append(audio)

inputs_list = []
for audio in audio_list:
    input_val = logfbank(audio, samplerate=16000, nfilt=26)
    input_val = (input_val - np.mean(input_val)) / np.std(input_val)
    inputs_list.append(input_val)

test_inputs = tf.ragged.constant([i for i in inputs_list], dtype=np.float32)
test_seq_len = tf.cast(test_inputs.row_lengths(), tf.int32)
test_inputs = test_inputs.to_tensor(default_value=1e+10)


In [26]:
with open('../data/alphabets_data.json', 'r', encoding='UTF-8') as alphabets_file:
    alphabets = load(alphabets_file)
with open('../data/train_labels.json', 'r', encoding='UTF-8') as label_file:
    labels = load(label_file)

In [27]:
prediction_list = []
decoded, _ = tf.nn.ctc_greedy_decoder(tf.transpose(test_model.predict(test_inputs), (1, 0, 2)), test_seq_len)
d = tf.sparse.to_dense(decoded[0], default_value=-1).numpy()
str_decoded = [''.join([alphabets['num_to_char'][str(x)]
                        for x in np.asarray(row) if x != -1]) for row in d]
for prediction in str_decoded:
    # Replacing blank label to none
    # s = s.replace(num_classes, '')
    # Replacing Spaces
    prediction = prediction.replace(
        alphabets['num_to_char']['0'], ' ')
    prediction_list.append(prediction)


In [28]:
i = 0
amount = 10
if amount > len(prediction_list): amount = len(prediction_list)
for key, value in labels.items():
    if(i > amount): break
    print('Original:')
    print(value)
    print('Model Transcription:')
    print(prediction_list[i])
    i += 1

Original:
የተለያዩ የትግራይ አውራጃ ተወላጆች ገንዘባቸውን አዋጥተው የልማት ተቋማትን እንዲመሰርቱ ትልማ አይፈቅድም
Model Transcription:
የተና የግራ ገቶር ላ ዱች ነዛባቸው ነወገጣ ለና ታጠጓናተሚእንዲናሳረቱ ትርማዋተድ
Original:
የጠመንጃ ተኩስ ተከፈተና አራት የኤርትራ ወታደሮች ተገደሉ
Model Transcription:
የጠመንጃ ተኩስ ተከፈተና አራት የኤርትራ ወታደሮች ተገደሉ
Original:
ላነሷቸው ጥያቄዎች የሰጡትን መልስ አቅርበነዋል
Model Transcription:
ባሩሳት ትጀርች የሰር ተቡ መስ አገርጋለዋ
Original:
እብዱ አስፋልቱ ላይ የኰለኰ ለውድንጋይ መኪና አላሳልፍአለ
Model Transcription:
እብዱ አስፋልቱ ላይ የኰለኰ ለውድንጋይ መኪና አላሳልፍአለ
Original:
ጠጁን ኰመኰመ ኰመኰመና ሚስቱን ሲያሰቃያት አደረ
Model Transcription:
ጠጁን ኰመኰመ ኰመኰመና ሚስቱን ሲያሰቃያት አደረ
Original:
ድንቹ በደንብ ስለተኰተኰተ በጥሩ ሁኔታ ኰረተ
Model Transcription:
ቤቹ በደን ስላተጐ ተባ ተ በተሩ ሁያየዳ ረተ
Original:
በድህነቱ ላይ ይህ ክፉ በሽታ ስለያዘው ሰውነቱ በጣም ኰሰሰ
Model Transcription:
በዴ ነቱ ላይ ይህህ ክፉ በሽታ ስላያለው ምናቱ በጣም ሰ
Original:
በሩን እንዲህ በሀይል አታንኳኲ ብዬ አልነበረም እንዴ
Model Transcription:
በሩን እንዲህ በሀይል አታንኳኲ ብዬ አልነበረም እንዴ
Original:
በለጠች የበየነ የበኩር ልጅ ነች
Model Transcription:
በለጠች የበየነ የበኩር ልጅ ነች
Original:
የቆላ ቁስልና ቁርጥማት በጣም አሰቃቂ በሽታዎች ናቸው
Model Transcription:
የቆላ ቁስልና ቁርጥማት በጣም አሰቃቂ በሽታዎ